In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
import itertools 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from ipywidgets import IntSlider, Output
from IPython.display import clear_output
import matplotlib.colors as mcolors
import h5py
import re
import matplotlib
import ipysheet
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.leap_tools import ccapt_tools

In [2]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [3]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

$$\textbf{You can specify which dataset to use in below block}$$

In [14]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_main_path = os.path.dirname(dataset_main_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/ions_selection/'
variables.result_data_name = 'range_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/ions_selection/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [15]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: T:/Monajem/physics_atom_probe_data/data_115_Jul-27-2022_17-44_Powersweep3/data_115_Jul-27-2022_17-44_Powersweep3
The dataset name after saving is: range_data_mc_calibration_cropped_data_115_Jul-27-2022_17-44_Powersweep3
The figures will be saved on the path: T:/Monajem/physics_atom_probe_data/data_115_Jul-27-2022_17-44_Powersweep3/data_115_Jul-27-2022_17-44_Powersweep3//ions_selection/
Total number of Ions: 522569


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,104.673501,105.185703,7640.335449,1899.187744,0,0.0,2121.0,-1.641570,-1.091160,0,0
1,0.0,0.0,0.0,102.478010,102.041997,7640.335449,1899.187744,0,0.0,2092.0,1.949420,1.261720,0,0
2,0.0,0.0,0.0,102.441867,102.313800,7640.335449,1899.187744,0,0.0,2095.0,-1.295210,1.965810,0,0
3,0.0,0.0,0.0,95.167228,95.468858,7640.335449,1899.187744,0,0.0,2019.0,-1.398370,-0.351357,0,0
4,0.0,0.0,0.0,126.353825,126.660767,7640.335449,1899.187744,0,0.0,2342.0,-3.353750,1.357700,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522564,0.0,0.0,0.0,87.356974,87.757034,6877.986328,2959.105713,0,0.0,2047.0,-2.509740,-0.313556,0,0
522565,0.0,0.0,0.0,376.895244,375.831645,6877.986328,2959.105713,0,0.0,4222.0,3.209090,-0.665806,0,0
522566,0.0,0.0,0.0,112.877694,112.597286,6877.986328,2959.105713,0,0.0,2307.0,1.114920,0.893752,0,0
522567,0.0,0.0,0.0,103.550324,104.025348,6877.986328,2959.105713,0,0.0,2225.0,0.236217,-2.371800,0,0


In [6]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc_c', 'mc_c'), ('tof_c', 'tof_c')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=1), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=widgets.Dropdown(options=[('True', True), ('False', False)]));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_12084\1242485591.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [55]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=50, description='peak prominance:')
distance=widgets.IntText(value=1, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('True', True), ('False', False)],
    description='plot peak:'
)

def hist_plot_p(variables, plot):
    
    with out:
        clear_output(True)
        # clear the peak_idx
        variables.peaks_idx = []
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_p], variables, bin_size_p, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_p, peaks_find=True, peaks_find_plot=plot_peak_p, plot=True, prominence=prominence_p,
              distance=None, h_line=False, selector='peak', fast_hist=True, fig_name=index_fig_p, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})

def hist_plot_r(variables, plot):
    
    with out:
        clear_output(True)
        bin_size_r = bin_size.value
        prominence_r = prominence.value
        distance_r = distance.value
        lim_tof_r = lim_tof.value
        percent_r = percent.value
        index_fig_r = index_fig.value
        plot_peak_r = plot_peak.value
        print('=============================')
        print('Press left click to draw a line')
        print('Press right click to remove a line')
        print('Press r to remove all the line')
        print('Hold shift and use mouse scroll for zooming on x axis')
        print('Hold ctrl and left mouse bottom to move a line')
        print('=============================')
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_r], variables, bin_size_r, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_r, peaks_find=True, peaks_find_plot=plot_peak_r, plot=True, prominence=prominence_r,
              distance=None, h_line=False, selector='range', fast_hist=True, fig_name=index_fig_r, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})
        
def hist_plot(variables, plot):
    with out:
        clear_output(True)
        bin_size_r = bin_size.value
        prominence_r = prominence.value
        distance_r = distance.value
        lim_tof_r = lim_tof.value
        percent_r = percent.value
        index_fig_r = index_fig.value
        plot_peak_r = plot_peak.value
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_r], variables, bin_size_r, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_r, peaks_find=True, peaks_find_plot=plot_peak_r, plot=True, prominence=prominence_r,
              distance=None, h_line=True, selector=None, fast_hist=True, fig_name=index_fig_r, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})

In [65]:
# element calculate
peak_val = widgets.FloatText(value=1.1, description='peak value:')
charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)
aboundance_threshold = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)
num_element = widgets.IntText(value=5, description='num element:')
# formula calculate
formula_m = widgets.Text(
    value='{12}C1{16}O2',
    placeholder='Type a formula  {12}C1{16}O2',
    description='Isotope formula:',
    disabled=False
)

molecule_charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

aboundance_threshold_molecule = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)

# molecule create
formula_com = widgets.Text(
    value='',
    placeholder="Elements ','",
    description='Elements:',
    disabled=False
)
complexity = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='complexity:'
)

charge_com = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

aboundance_threshold_com = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)



In [66]:
plot_button_p = widgets.Button(
    description='plot hist',
)

plot_button = widgets.Button(
    description='plot hist',
)


find_elem_button = widgets.Button(
    description='find element',
)
find_mole_button = widgets.Button(
    description='find molecule',
)

formula_button = widgets.Button(
    description='manual formula',
)

add_ion_button = widgets.Button(
    description='add ion',
)
romove_ion_button = widgets.Button(
    description='remove ion',
)
show_color = widgets.Button(
    description='show color',
)
change_color = widgets.Button(
    description='change color',
)

color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
mass_unknown = widgets.FloatText(value=0.0, description='unknown mass:')
@plot_button_p.on_click
def plot_on_click_p(b, plot=True):
    hist_plot_p(variables, plot)
    
    
@plot_button.on_click
def plot_on_click_b(b, plot=True):
    hist_plot(variables, plot)
    
@find_elem_button.on_click
def vol_on_click(b,):
    with output2:
        peak_val_s = peak_val.value
        charge_s = charge.value
        num_element_s = num_element.value
        aboundance_threshold_s = aboundance_threshold.value
        clear_output(True)
        df = ion_selection.find_close_element(peak_val_s, num_element_s, aboundance_threshold_s, charge=charge_s, variables=variables)
        display(df)
    
@find_mole_button.on_click
def mol_on_click(b,):
    with output2:
        if formula_com.value == '':
            print("Input is empty. Type at least two elements.")
        else:
            df = ion_selection.molecule_create(formula_com.value.split(','), complexity.value, molecule_charge.value, aboundance_threshold_molecule.value)
            clear_output(True)
            display(df)
    
@formula_button.on_click
def manual_formula(b,):
    with output2:
        if formula_m.value == '':
            print("Input is empty. Type the formula.")
        else:
            df = ion_selection.molecule_manual(formula_m.value, molecule_charge.value, latex=True, variables=variables)
            clear_output(True)
            display(df)

@add_ion_button.on_click
def add_ion_to_range_dataset(b,):
    ion_selection.ranging_dataset_create(variables, row_index.value, mass_unknown.value)
    with output3:
        clear_output(True)
        display(variables.range_data)
@romove_ion_button.on_click
def remove_ion_to_range_dataset(b,):
    if len(variables.range_data) >= 1:
        variables.range_data = variables.range_data.drop(len(variables.range_data) - 1)
        with output3:
            clear_output(True)
            display(variables.range_data)
        
@show_color.on_click
def show_color_ions(b,):
    with output3:
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
@change_color.on_click
def change_color_m(b,):
    with output3:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

        # Create "Next" and "Previous" buttons
start_button = widgets.Button(description="Start")
next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")
reset_h_line_button = widgets.Button(description="Reset")
all_peaks_button = widgets.Button(description="Add all peaks")


# Define button click events
@start_button.on_click
def start_peak(_):
    print('=============================')
    print('Press left click to draw a line')
    print('Press right click to remove a line')
    print('Press r to remove all the line')
    print('Press a to automatically draw lines')
    print('Hold shift and use mouse scroll for zooming on x axis')
    print('Hold ctrl and left mouse bottom to move a line')
    print('=============================')
    variables.peaks_index = 0
    peak = variables.peak[variables.peaks_idx[variables.peaks_index]]
    peak_val.value = peak
    print('peak idc:', variables.peaks_index, 'Peak location:', peak)
    mask = (variables.mc_c < peak + 5) & (variables.mc_c > peak - 5)
    mc = np.copy(variables.mc_calib)
    peaks = np.copy(variables.peak)
    variables.mc_c = variables.mc_c[mask]
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak = peaks
@next_button.on_click
def next_peak(_):
    variables.peaks_index = variables.peaks_index + 1
    peak = variables.peak[variables.peaks_idx[variables.peaks_index]]
    peak_val.value = peak
    print('peak idc:', variables.peaks_index, 'Peak location:', peak)
    mask = (variables.mc_c < peak + 5) & (variables.mc_c > peak - 5)
    mc = np.copy(variables.mc_calib)
    peaks = np.copy(variables.peak)
    variables.mc_c = variables.mc_c[mask]
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak = peaks
@prev_button.on_click
def prev_peak(_):
    variables.peaks_index = variables.peaks_index - 1
    peak = variables.peak[variables.peaks_idx[variables.peaks_index]]
    peak_val.value = peak
    print('peak idc:', variables.peaks_index, 'Peak location:', peak)
    mask = (variables.mc_c < peak + 5) & (variables.mc_c > peak - 5)
    mc = np.copy(variables.mc_calib)
    peaks = np.copy(variables.peak)
    variables.mc_c = variables.mc_c[mask] 
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak = peaks

@reset_h_line_button.on_click
def rest_h_line(_):
    variables.h_line_pos = []
@all_peaks_button.on_click
def select_all_peaks(_):
    variables.peaks_idx = list(range(0,len(variables.peak)))
    
tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button_p, all_peaks_button])])
tab2 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[VBox(children=[start_button, next_button, prev_button, reset_h_line_button])])])
tab3 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button])])
tab4 = VBox(children=[HBox(children=[VBox(children=[peak_val, num_element, charge, aboundance_threshold, 
                      find_elem_button]),  VBox(children=[formula_com, complexity, charge_com, aboundance_threshold_com, find_mole_button]),
                      VBox(children=[formula_m, molecule_charge, formula_button])]), VBox(children=[row_index, color_picker, mass_unknown, add_ion_button, romove_ion_button, 
                                                                                                                                      show_color, change_color])])

tabs1 = widgets.Tab(children=[tab1, tab2, tab3])
tabs2 = widgets.Tab(children=[tab4])
tabs1.set_title(0, 'peak finder')
tabs1.set_title(1, 'rangging')
tabs1.set_title(2, 'rangged')
tabs2.set_title(0, 'element finder')
# Create two Output widgets to capture the output of each plot
out = Output()
output2 = Output()
output3 = Output()

# Create an HBox to display the buttons side by side
buttons_layout = widgets.HBox([tabs1, tabs2])

# Create a VBox to display the output widgets below the buttons
output_layout_h = widgets.HBox([out, output3])
output_layout = widgets.VBox([output_layout_h, output2])


# Display the buttons and the output widgets
display(buttons_layout, output_layout)


with output3:
    display(variables.range_data)

In [67]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Create a sample DataFrame with mixed data types
data = {'Column1': [1, 2.5, 'Text'],
        'Column2': [4, 5.6, 'More Text']}
df = pd.DataFrame(data)

def make_widget(index, col):
    cell_value = df.iloc[index, col]
    if isinstance(cell_value, int):
        widget = widgets.IntText(value=cell_value)
    elif isinstance(cell_value, float):
        widget = widgets.FloatText(value=cell_value)
    else:
        widget = widgets.Text(value=str(cell_value))
    widget.observe(update_dataframe, names='value')
    return widget

def update_dataframe(change):
    description = change.owner.description
    index, col = [int(x) for x in description.split(' ')[-1][1:-1].split(',')]
    if isinstance(df.iloc[index, col], int):
        df.iloc[index, col] = int(change.new)
    elif isinstance(df.iloc[index, col], float):
        df.iloc[index, col] = float(change.new)
    else:
        df.iloc[index, col] = str(change.new)

rows, cols = df.shape
widgets_grid = [[make_widget(i, j) for j in range(cols)] for i in range(rows)]
column_labels = [widgets.Text(value=col) for col in df.columns]

# Insert column labels as the first row in the widgets grid
widgets_grid.insert(0, column_labels)

# Create a VBox for each row and organize them in an HBox
hboxes = [widgets.HBox(row) for row in widgets_grid]
interactive_grid = widgets.VBox(hboxes)
display(interactive_grid)


In [29]:
def mc_hist_plot(bin_size, lim_mc, hist_color_range, figname):
    peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(variables.mc_c[variables.mc_c < lim_mc], variables, bin=bin_size, range_data=variables.range_data, ranging=True, hist_color_range=hist_color_range, adjust_label=False, peaks_find=True, peaks_find_plot=True, plot=True, label='mc', fig_name=figname)
   
interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)])
                , figname=widgets.Text(value='ranged'));

C:\Users\APTUser\AppData\Local\Temp\ipykernel_28960\116442335.py:4: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)])


interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=400, description='lim_mc'), …

In [20]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,peak_count,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,0.923670,0.923670,0.823657,#b2aa2d,151,H,1,1,+
1,${}^{2}H^{+}$,2.010000,1.523748,2.323852,1.223709,#b2aa2d,35,H,1,2,+
2,${}^{13}C^{+}$,12.990000,13.825342,13.625316,15.325537,#daa460,214,C,1,13,+
3,${}^{18}O^{+}$,18.313333,18.125900,17.325796,20.826250,#1f963a,364,O,1,18,+
4,${}^{48}Ti^{2+}$,23.975000,23.926651,25.026794,22.626483,#531b9c,16337,Ti,1,48,2+
5,${}^{48}Ti^{+}$,48.303333,48.329814,51.630241,39.328647,#531b9c,151,Ti,1,48,+
6,${}^{184}W^{2+}$,93.480000,92.735568,95.035866,90.235244,#ab70d5,412,W,1,184,2+
7,${}^{184}W^{+}$,180.950000,183.647349,180.946999,191.648386,#ab70d5,49,W,1,184,+


In [21]:
variables.range_data.dtypes

ion            object
mass          float64
mc            float64
mc_low        float64
mc_up         float64
color          object
peak_count      int64
element        object
complex         int64
isotope         int64
charge         object
dtype: object

In [22]:
# save the new data
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)

In [23]:
# save data in csv format
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)